# DLO-JZ Fully Sharded Data parallelism - Jour 4

Utilisation de la FSDP sur un modèle de langue **Llama 3.2 3B**.

![Monstertruck](./images/MonsterTruck.png)


## Objet du notebook

Le but de ce *notebook* est d'optimiser un code d'apprentissage d'un modèle *Llama 3.2* sur un dataset de roleplay *Imagenet* :
* Passage de DDP à FSDP
* FSDP optimisée grâce au wrap récursif
* Bonus : Application de la compilation PyTorch par dessus la FSDP


Les cellules dans ce *notebook* ne sont pas prévues pour être modifiées, sauf rares exceptions indiquées dans les commentaires. Les TP se feront en modifiant le code `fsdp.py`.

Les directives de modification seront marquées par l'étiquette **TODO :** dans le *notebook* suivant.
 
Les solutions sont présentes dans le répertoire `solutions/`.

*Notebook rédigé par l'équipe assistance IA de l'IDRIS, octobre 2024*

---

### Environnement de calcul

Les fonctions *python* de gestion de queue Slurm dévelopées par l'IDRIS et les fonctions dédiées à la formation DLO-JZ sont à importer.

Le module d'environnement pour les *jobs* et la taille des images sont fixés pour ce *notebook*.

**TODO :** choisir un *pseudonyme* pour vous différencier dans la queue Slurm et dans les outils collaboratifs pendant la formation.

In [ ]:
from idr_pytools import display_slurm_queue, gpu_jobs_submitter, search_log
from dlojz_tools import controle_technique, compare, GPU_underthehood, plot_accuracy, lrfind_plot, pipe_memory, turbo_profiler, comm_profiler
MODULE = 'pytorch-gpu/py3/2.4.0'
account = 'for@a100'
name = 'NATHOUNET'

---
### Gestion de la queue Slurm

Cette partie permet d'afficher et de gérer la queue Slurm.

Pour afficher toute la queue *utilisateur* :

In [ ]:
#display_slurm_queue(name)

**Remarque**: Cette fonction utilisée plusieurs fois dans ce *notebook* permet d'afficher la queue de manière dynamique, rafraichie toutes les 5 secondes. Cependant elle ne s'arrête que lorsque la queue est vide. Si vous désirez reprendre la main sur le *notebook*, il vous suffira d'arrêter manuellement la cellule avec le bouton *stop*. Cela a bien sûr aucun impact sur le *scheduler* Slurm. Les *jobs* ne seront pas arrêtés.

Si vous voulez arrêter des *jobs* dans la queue :
* Annuler tous vos *jobs* dans la queue (décommenter la ligne suivante)
* Annuler un *job* dans votre queue (décommenter la ligne suivante et ajouter le numéro du *job* à la fin de la ligne)

In [ ]:
#!scancel -u $USER

---
### Différence de scripts <a id='diff_scripts'></a>

Pour le *debug* ou pour comparer son code avec les solutions mises à disposition, la fonction suivante permet d'afficher une page html contenant un différentiel de fichiers texte.

In [ ]:
s1 = "fsdp.py"
s2 = "./solutions/fsdp_2.py"
compare(s1, s2)

Voir le résultat du différentiel de fichiers sur la page suivante (attention au spoil !) :

[compare.html](compare.html)

---
# Première exécution en DDP

In [ ]:
!cp solutions/fsdp_0.py fsdp.py

Prenez connaissance du script **fsdp.py**. C'est un fine-tuning d'un modèle de langue Llama 3.2 à 3 milliards de paramètres sur un dataset de Roleplay récupéré sur HuggingFace.

La structure entre ce script et celui lié à la computer vision des autres jours est très similaire.

_Note : on utilise les mêmes visualisations que dans les autres TPs donc les schémas peuvent parler d'images, mais dans notre contexte de NLP, la dimension du batch ne fait pas référence à des images mais plutôt aux nombres de séquences._

In [ ]:
n_gpu = 1
command = f'fsdp.py --batch-size 4 --num-workers 2 --seq-len 512 --test'
jobid = gpu_jobs_submitter(command, n_gpu, MODULE, name=name,
                   account=account, time_max='00:10:00', qos="qos_gpu_a100-dev")
print(f'jobid = {jobid}')

In [ ]:
display_slurm_queue(name)

In [ ]:
#jobid = ['902284']

In [ ]:
controle_technique(jobid)

In [ ]:
turbo_profiler(jobid)

### Test d'occupation mémoire

Afin de mesurer l'impact de la taille de batch sur l'occupation mémoire et sur le *throughput*, la cellule suivante permet de soumettre plusieurs *jobs* avec des tailles de *batch* croissantes. Dans les cas où la mémoire est saturée et dépasse la capacité du GPU, le système renverra une erreur *CUDA Out of Memory*.

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

In [ ]:
n_gpu = 4
batch_sizes = [1, 2, 4, 8]
command = [f'fsdp.py --batch-size {batch_size} --num-workers 2 --seq-len 512 --test'
          for batch_size in batch_sizes]
jobids = gpu_jobs_submitter(command, n_gpu, MODULE, name=name,
                   account=account, time_max='00:10:00', constraint='a100', qos="qos_gpu_a100-dev")
print(f'jobids = {jobids}')

In [ ]:
display_slurm_queue(name)

In [ ]:
GPU_underthehood(jobids)

---
# Passage au Fully Sharded Data Parallelism

**TODO**: Remplacez le Distributed Data Parallelism par le Fully Sharded Data Parallelism. Ce n'est qu'un simple wrapper et demande peu de modifications. Indice : ctrl-F de "#### Distribute the Model" pour repérer l'endroit où faire ça.

**Important**: Dans la documentation, vous pouvez voir un paramètre _auto_wrap_policy_, il est abordé dans la section suivante donc laissez le non précisé pour l'instant.

FSDP est un wrapper très haut niveau qui fait toutes les communications de manière cachée pour faciliter son utilisation. C'est son grand avantage par rapport à DeepSpeed.

![fsdp](images/fsdp.png)

In [ ]:
n_gpu = 4
command = f'fsdp.py --batch-size 4 --num-workers 2 --seq-len 512 --test'
jobid = gpu_jobs_submitter(command, n_gpu, MODULE, name=name,
                   account=account, time_max='00:10:00', qos="qos_gpu_a100-dev")
print(f'jobid = {jobid}')

In [ ]:
display_slurm_queue(name)

In [ ]:
#jobid = ['902284']

In [ ]:
controle_technique(jobid)

In [ ]:
pipe_memory(jobid)

### Test d'occupation mémoire

Afin de mesurer l'impact de la taille de batch sur l'occupation mémoire et sur le *throughput*, la cellule suivante permet de soumettre plusieurs *jobs* avec des tailles de *batch* croissantes. Dans les cas où la mémoire est saturée et dépasse la capacité du GPU, le système renverra une erreur *CUDA Out of Memory*.

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

In [ ]:
n_gpu = 4
batch_sizes = [2, 4, 8, 16]
command = [f'fsdp.py --batch-size {batch_size} --num-workers 2 --seq-len 512 --test'
          for batch_size in batch_sizes]
jobids = gpu_jobs_submitter(command, n_gpu, MODULE, name=name,
                   account=account, time_max='00:10:00', constraint='a100', qos="qos_gpu_a100-dev")
print(f'jobids = {jobids}')

In [ ]:
#jobids = ['299137', '299138', '299139', '299140', '299141']

In [ ]:
display_slurm_queue(name)

In [ ]:
GPU_underthehood(jobids)

### Contrôle technique de la configuration optimale

In [ ]:
controle_technique([jobids[-2]])

---
# Optimisation de la FSDP en l'appliquant récursivement

En utilisant la FSDP sans l'argument _auto_wrap_policy_, alors tous les paramètres du modèle sont transmis en une seule fois plutôt que quand ils sont nécessaires. On pourrait imaginer transmettre d'abord les paramètres des premières couches (dont on a besoin rapidement), puis ensuite les autres.

Le paramètre _auto_wrap_policy_ permet de faire ça. Il est puissant car il permet de maximiser le recouvrement entre les calculs et les communications et minimise la mémoire nécessaire.

**TODO**: Ajoutez le paramètre _auto_wrap_policy_. Indice : Pour les transformers, il est de coutume de wrapper chaque block de transformer indépendamment (signature de la méthode _transformer_auto_wrap_policy_ [ici](https://github.com/pytorch/pytorch/blob/8f3efb8797b7a2dbd958bf625374985793ed5035/torch/distributed/fsdp/wrap.py#L307)).

In [ ]:
n_gpu = 4
command = f'fsdp.py --batch-size 4 --num-workers 2 --seq-len 512 --test'
jobid = gpu_jobs_submitter(command, n_gpu, MODULE, name=name,
                   account=account, time_max='00:10:00', qos="qos_gpu_a100-dev")
print(f'jobid = {jobid}')

In [ ]:
display_slurm_queue(name)

In [ ]:
#jobid = ['902284']

In [ ]:
controle_technique(jobid)

Vous pouvez consulter les logs de l'entraînement. On affiche le modèle, et on peut par conséquent voir l'impact de l'argument _auto_wrap_policy_.

### Test d'occupation mémoire

Afin de mesurer l'impact de la taille de batch sur l'occupation mémoire et sur le *throughput*, la cellule suivante permet de soumettre plusieurs *jobs* avec des tailles de *batch* croissantes. Dans les cas où la mémoire est saturée et dépasse la capacité du GPU, le système renverra une erreur *CUDA Out of Memory*.

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

In [ ]:
n_gpu = 4
batch_sizes = [4, 8, 12, 16, 32]
command = [f'fsdp.py --batch-size {batch_size} --num-workers 2 --seq-len 512 --test'
          for batch_size in batch_sizes]
jobids = gpu_jobs_submitter(command, n_gpu, MODULE, name=name,
                   account=account, time_max='00:10:00', constraint='a100', qos="qos_gpu_a100-dev")
print(f'jobids = {jobids}')

In [ ]:
#jobids = ['300073', '300074', '300075', '300076', '300077']

In [ ]:
display_slurm_queue(name)

In [ ]:
GPU_underthehood(jobids)

### Contrôle technique de la configuration optimale

In [ ]:
controle_technique(jobids[-3])

---
# Bonus : torch.compile par dessus FSDP

**TODO**: Appliquez la compilation par PyTorch de votre modèle.

**Indice**: ctrl-F de "#### JIT" pour trouver où faire ça.

In [ ]:
n_gpu = 4
command = f'fsdp.py --batch-size 16 --num-workers 2 --seq-len 512 --test --compile'
jobid = gpu_jobs_submitter(command, n_gpu, MODULE, name=name,
                   account=account, time_max='00:10:00', qos="qos_gpu_a100-dev")
print(f'jobid = {jobid}')

In [ ]:
#jobid = ['902284']

In [ ]:
display_slurm_queue(name)

In [ ]:
controle_technique(jobid)

### Test d'occupation mémoire

Afin de mesurer l'impact de la taille de batch sur l'occupation mémoire et sur le *throughput*, la cellule suivante permet de soumettre plusieurs *jobs* avec des tailles de *batch* croissantes. Dans les cas où la mémoire est saturée et dépasse la capacité du GPU, le système renverra une erreur *CUDA Out of Memory*.

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

In [ ]:
n_gpu = 4
batch_sizes = [8, 12, 16, 20, 24]
command = [f'fsdp.py --batch-size {batch_size} --num-workers 2 --seq-len 512 --test --compile'
          for batch_size in batch_sizes]
jobids = gpu_jobs_submitter(command, n_gpu, MODULE, name=name,
                   account=account, time_max='00:10:00', constraint='a100', qos="qos_gpu_a100-dev")
print(f'jobids = {jobids}')

In [ ]:
#jobids = ['300403', '300404', '300406', '300407', '300408', '300409']

In [ ]:
display_slurm_queue(name)

In [ ]:
GPU_underthehood(jobids)

### Contrôle technique de la configuration optimale

In [ ]:
controle_technique(jobids[-2])

_torch.compile_ est encore très nouveau et il peut arriver qu'un modèle ne puisse pas être converti. Plusieurs backends sont disponibles (voir documentation officielle). Dans les cas des modèles les plus exotiques, la compilation peut tout simplement échoué. C'est tellement bas niveau qu'il est bien possible qu'on ne puisse rien y faire, c'est juste lié au fait que _torch.compile_ est relativement nouveau. À garder à l'esprit cependant, car cela peut augmenter de 50%, voire parfois 100% le throughput de votre modèle.

![Commentaires](images/cedez.png "La suite correspond aux annexes, vous etes arrivé à bout du TP, BRAVO")


---